In [1]:
# load packages
import os
import sys
import re
import numpy as np
import tensorflow as tf
from sklearn.datasets import load_boston
tf.__version__

'1.12.0'

In [2]:
# load data
boston = load_boston()
features = np.array(boston.data)
labels = np.array(boston.target)
features.shape, labels.shape

((506, 13), (506,))

In [3]:
labels = labels.reshape((len(labels), 1)) # reshape rank 1 array to have 1 dim
labels.shape

(506, 1)

In [4]:
features[0]

array([6.320e-03, 1.800e+01, 2.310e+00, 0.000e+00, 5.380e-01, 6.575e+00,
       6.520e+01, 4.090e+00, 1.000e+00, 2.960e+02, 1.530e+01, 3.969e+02,
       4.980e+00])

In [5]:
labels[0]

array([24.])

In [6]:
mean = np.mean(features, axis=0) # feature wise mean
std = np.std(features, axis=0) # feature wise standard deviation
# normalize feature now
features = (features - mean) / std
features.shape

(506, 13)

In [7]:
features[0]

array([-0.41978194,  0.28482986, -1.2879095 , -0.27259857, -0.14421743,
        0.41367189, -0.12001342,  0.1402136 , -0.98284286, -0.66660821,
       -1.45900038,  0.44105193, -1.0755623 ])

In [8]:
random_indices = np.random.rand(len(features)) < 0.80 # random indexes for splitting
# splti data
train_x = features[random_indices]
train_y = labels[random_indices]
val_x = features[~random_indices]
val_y = labels[~random_indices]

In [9]:
# set dimension
num_features = features.shape[1]
num_features

13

In [10]:
# create placeholders for input
x = tf.placeholder(dtype=tf.float32, shape=(None, num_features), name="feature_matrix") # num_samples x num_features
y = tf.placeholder(dtype=tf.float32, shape=(None, 1), name="target_vector") # num_sampels X 1

In [11]:
# define weight for the model
w = tf.Variable(tf.zeros(dtype=tf.float32, shape=(1, num_features)))

In [12]:
# define bais
b = tf.Variable(tf.zeros(dtype=tf.float32, shape=(1, num_features)))

In [13]:
# linear model
linear_model = tf.add(tf.multiply(w, x), b)

In [14]:
loss_function = tf.reduce_mean(tf.square(linear_model - y)) # define loss function

### Using Inbuilt Optimizer

In [15]:
trainer = tf.train.AdamOptimizer(learning_rate=0.01).minimize(loss_function) # trainer

In [16]:
# run the mode
with tf.Session() as sess:
    epochs = 10000
    print("[Training...]\n")
    sess.run(tf.global_variables_initializer()) # initialize variables
    # run the optimizer
    for i in range(epochs+1):
        sess.run(trainer, feed_dict={x:train_x, y:train_y}) # train and update weights and bais
        c = sess.run(loss_function, feed_dict={x:train_x, y:train_y}) # get the loss for current w and b
        if i % 1000 == 0:
            print("Step", i, ":", end="\t")
            print("Train Loss: ", c)
    print("Validation Loss: ", sess.run(loss_function, feed_dict={x:val_x, y:val_y}))
    print("Final Weight:", sess.run(w))
    print("Final Bias:", sess.run(b))
    print()

[Training...]

Step 0 :	Train Loss:  583.68005
Step 1000 :	Train Loss:  245.65903
Step 2000 :	Train Loss:  110.3113
Step 3000 :	Train Loss:  71.29766
Step 4000 :	Train Loss:  66.01469
Step 5000 :	Train Loss:  65.832054
Step 6000 :	Train Loss:  65.83143
Step 7000 :	Train Loss:  65.831436
Step 8000 :	Train Loss:  65.83145
Step 9000 :	Train Loss:  65.83146
Step 10000 :	Train Loss:  65.83145
Validation Loss:  72.300446
Final Weight: [[-3.433359   3.2114494 -4.4950027  1.3011824 -3.9602838  6.361422
  -3.3996427  2.2104735 -3.5277543 -4.3166003 -4.481557   3.0122895
  -6.688876 ]]
Final Bias: [[22.403582 22.365278 22.244686 22.401722 22.311848 22.432194 22.349125
  22.327875 22.364016 22.301714 22.412426 22.37409  22.375612]]



### Using Implemented Optimizer

In [17]:
grad_w, grad_b = tf.gradients(xs=[w, b], ys=loss_function) # compute gradient

In [18]:
# update weights and bias
new_w = w.assign(w - 0.01 * grad_w) # assign new value
new_b = b.assign(b - 0.01 * grad_b)

In [21]:
# run the mode
with tf.Session() as sess:
    epochs = 10000
    print("[Training...]\n")
    sess.run(tf.global_variables_initializer()) # initialize variables
    # run the optimizer for 1000 iterations
    for i in range(epochs+1):
        _, _, c = sess.run([new_w, new_b, loss_function], feed_dict={x:train_x, y:train_y})
        if i % 1000 == 0:
            print("Step", i, ":", end="\t")
            print("Train Loss: ", c)
    print("Validation Loss: ", sess.run(loss_function, feed_dict={x:val_x, y:val_y}))
    print("Final Weights:", sess.run(w))
    print("Final Bias:", sess.run(b))

[Training...]

Step 0 :	Train Loss:  584.2093
Step 1000 :	Train Loss:  89.48396
Step 2000 :	Train Loss:  66.91366
Step 3000 :	Train Loss:  65.88108
Step 4000 :	Train Loss:  65.8337
Step 5000 :	Train Loss:  65.83154
Step 6000 :	Train Loss:  65.83145
Step 7000 :	Train Loss:  65.83143
Step 8000 :	Train Loss:  65.83143
Step 9000 :	Train Loss:  65.83143
Step 10000 :	Train Loss:  65.83143
Validation Loss:  72.30171
Final Weights: [[-3.4332843  3.2113822 -4.495178   1.3011379 -3.9602969  6.3612566
  -3.3995965  2.2105646 -3.5276856 -4.3165474 -4.4813957  3.0122185
  -6.688724 ]]
Final Bias: [[22.402983 22.364681 22.244083 22.401127 22.311249 22.431597 22.348528
  22.327276 22.363417 22.301117 22.411829 22.373493 22.375015]]
